In [110]:
# import library
import pickle, os
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import copy
from scipy.stats.mstats import winsorize
from statsmodels.formula.api import ols

os.chdir('C:\\Users\\lyoo9\\_Akros')
# # Data Download
with open('FS_손익계산서보정본', 'rb') as f:
    data = pickle.load(f)

with open('Final_FS_ind', 'rb') as f:
    ind = pickle.load(f)

# 특정 자산만 뽑아서 csv로 정리하기
def get_item(concept_id):
    item = []
    item_ind = []
    # check id's address
    id_address = None
    for i in range(len(data)):
        for j in range(3):
            try:
                if concept_id in data[i][j].iloc[:,0].values:
                    id_address = j
            except AttributeError:
                pass
        if id_address != None:
            break
            
    for i in range(len(data)):
        if type(data[i][id_address]) == type(data[0][id_address]):
            item.append(data[i][id_address][data[i][id_address][data[i][id_address].columns[0]] == concept_id].iloc[:1])
            if len(data[i][id_address][data[i][id_address][data[i][id_address].columns[0]] == concept_id]) > 0:
                item_ind.append(ind[i])
    df_item = pd.concat(item)
    date = list(filter(lambda x: x[1] == ('연결재무제표',), df_item.columns))
    if id_address ==0:
        date_clean = list(map(lambda x: x[0], df_item[date].columns))
        df_item[date].columns = date_clean
        df_item2 = df_item[date]
        df_item2.columns = date_clean
        date_shifted = list(map(lambda x: shift_q(x), date_clean))
        df_item3 = df_item2
        df_item3.columns = date_shifted
        df_item4 = df_item3.groupby(level=0, axis=1).last()
        df_item3 = df_item4
        date_shifted2 = list(map(lambda x: shift_d(x), df_item4.columns))
        df_item3.columns = date_shifted2
        df_item5 = df_item3.groupby(level=0, axis=1).last()
        corp_code = list(map(lambda x:x[2], item_ind))
        df_item5['corp_code'] = corp_code
        df_item5 = df_item5.set_index('corp_code')
    else:
        date_clean = list(map(lambda x: x[0], df_item[date].columns))
        df_item[date].columns = date_clean
        df_item2 = df_item[date]
        df_item2.columns = date_clean
        date_shifted = list(map(lambda x: shift_flow_date(x), date_clean))
        df_item2.columns = date_shifted
        df_item3 = df_item2.groupby(level=0, axis=1).last()
        corp_code = list(map(lambda x:x[2], item_ind))
        df_item3['corp_code'] = corp_code
        df_item3 = df_item3.set_index('corp_code')
        df_item5 = df_item3
    return df_item5

def shift_q(date):
    if date[4:6]=='01' or date[4:6]=='02':
        date = date[:4] + '0331'
    elif date[4:6]=='04' or date[4:6]=='05':
        date = date[:4] + '0630'
    elif date[4:6]=='07' or date[4:6]=='08':
        date = date[:4] + '0930'
    elif date[4:6]=='10' or date[4:6]=='11':
        date = date[:4] + '1231'
    return date

def shift_d(date):
    # 케이스 두개 밖에 안되니까 그냥 수동으로 따져주자.
    if date[6:]=='01':
        date = date[:6] + '31'
    elif date[6:]=='29':
        date = date[:6] + '30'
    return date

def shift_flow_date(date):
    # 우선은 연간 데이터만 활용하도록 만들어놓아도 될 듯? 나중에 분기별로 뽑을 수 있는 옵션을 추가해야 할 것 같다.
    if date[4:6]=='01' and date[-4:-2]=='12':
        date = date[-8:-2] + '31'
    else:
        date = None
    return date

def get_item_on_date(item, date):
    return get_item(item)[date]

def get_item_for_period(item, date_from, date_to, term = 'Y'):
    # term은 Y, H, Q의 세 가지 옵션으로, 연도별, 반기별, 분기별 데이터를 얻어올 수 있도록 하는 옵션이다.
    data = []
    item_data = get_item(item)
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(item_data[date_working])
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            try:
                data.append(item_data[date_working])
            except:
                try:
                    date_working = date_working[:6] + str(61 - int(date_working[6:]))
                    data.append(item_data[date_working])
                except KeyError:
                    print('해당 날짜의 데이터가 없습니다:', date_working)
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
                
    data = pd.concat(data, axis = 1)
    return data

os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')
def get_market_data_on_date(date, print_error = True):
    # 에러 메시지를 출력하고 싶지 않으면 print_error를 False로 설정
    try:
        data = pd.read_csv(date[2:4] + '_' + str(int(date[4:6])) + '_' + str(int(date[6:8])) + '.csv', encoding = 'cp949')
        data['종목코드'] = data['종목코드'].map(lambda x:str(x).zfill(6) if x != None else None)
        if data.isna()['종가'].sum() == len(data):
            raise FileNotFoundError
    except FileNotFoundError:
        if print_error:
            print('해당 날짜의 데이터가 존재하지 않습니다:', date)
        data = None
    return data

def get_market_cap_on_date(date, get_code = False):
    if get_code:
        data = get_market_data_on_date(date)[['종목코드','시장구분','시가총액']].set_index('종목코드')
        data.columns = [['시장구분','Market_Cap']]
    else:
        data = get_market_data_on_date(date)[['종목코드','시가총액']].set_index('종목코드')
        data.columns = [['Market_Cap']]
    data.index.rename('corp_code', inplace = True)
    return data

def last_market_date(date):
    # date의 기본형식은 'YYYYMM' : ex)'201212'. -> 'YYYYMMDD'도 가능하도록 코드 변경했음
    get_data = False
    day = 31
    while not get_data:
        if type(get_market_data_on_date(date[:6] + str(day), print_error = False)) != type(None):
            market_date = date[:6] + str(day)
            get_data = True
        else:
            day -=1
    return market_date

def find_next_market_date(date, today_opt = False):
    get_data = False
    if today_opt:
        market_date = pd.Timestamp(date)
    else:
        market_date = pd.Timestamp(date) + pd.Timedelta(days=1)
    while not get_data:
        if type(get_market_data_on_date(market_date.strftime('%Y%m%d'), print_error = False)) != type(None):
            get_data = True
        else:
            market_date += pd.Timedelta(days=1)
    
    return market_date.strftime('%Y%m%d')

def get_market_cap_for_period(date_from, date_to, term = 'Y', ind_match = True):
    # ind_match는 fundamental과 합치기 편하도록 실제 날짜가 아니라 월말 날짜를 표시하도록 하는 옵션
    data = []
    ind = []
    if term == 'Y':
        num = int(date_to[:4]) - int(date_from[:4]) +1
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            date_working = date_working[:2] + str(int(date_working[2:4]) + 1) + date_working[4:]
            
    elif term == 'H':
        num = int((int(date_to[:4]) - int(date_from[:4])) * 2 + (int(date_to[4:6]) - int(date_from[4:6]))/6 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+6 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0'+ str(int(date_working[4:6])-6)+'31'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 6).zfill(2) + '31'
                
    else:
        num = int((int(date_to[:4]) - int(date_from[:4])) * 4 + (int(date_to[4:6]) - int(date_from[4:6]))/3 + 1)
        date_working = date_from
        for i in range(num):
            data.append(get_market_cap_on_date(last_market_date(date_working), get_code = True))
            if ind_match:
                ind.append(date_working)
            else:
                ind.append(last_market_date(date_working))
            if int(date_working[4:6])+3 > 12:
                date_working = date_working[:2] + str(int(date_working[2:4]) + 1) +'0331'
            else:
                date_working = date_working[:4] + str(int(date_working[4:6]) + 3).zfill(2) + '31'
    cap_data = [x.iloc[:,1] for x in data]
    cat_data = [x.iloc[:,0] for x in data]
    cap_data = pd.concat(cap_data, axis =1)
    cap_data.columns = ind
    cat_data = pd.concat(cat_data, axis =1)
    cat_data.columns = ind
    return cap_data, cat_data

def get_market_category(data):
    market_cat = []
    for i in range(len(data.columns)):
        market_cat.append(get_market_cap_on_date(last_market_date(data.columns[i]), get_code = True).iloc[:,0])
    market_cat = pd.concat(market_cat, axis=1)
    market_cat.columns = data.columns
    return market_cat

def get_breakpoint(data, breakpoint):
    absolute_point = []
    rank = data.rank(pct=True)
    for i in range(len(breakpoint)):
        absolute_point.append((data[rank < breakpoint[i]].max() + data[rank > breakpoint[i]].min())/2)
    return absolute_point

def get_mask(data, market_cat = None, market_bp = 'All', breakpoint: list = [0.5]):
    # data는 mask의 기준이 되는 baseline data
    # market_bp는 'ALL', 'KOSPI', 'KOSDAQ'으로, 기준이 되는 breakpoint의 market을 의미
    # breakpoint는 masking의 구분점으로, list안에 float이 담긴 형태를 받음
    breakpoint = np.sort(breakpoint)
    mask_list = []
    
    data_ind = data.index
    if type(market_cat) != type(None):
        market_cat = market_cat[market_cat.index.map(lambda x: x in data.index)]
        for i in range(len(data)):
            if data_ind[i] not in market_cat.index:
                market_cat.loc[data_ind[i]] = None
        market_cat = market_cat.reindex(index = data.index)
    for i in range(len(data.columns)):
        mask = data.iloc[:,i] * 0 +1
        mask_np = -np.ones(len(mask))
        #rank = data.iloc[:,i].rank(pct = True)
        if market_bp == 'All':
            absolute_point = get_breakpoint(data.iloc[:,i], breakpoint)
        if market_bp == 'KOSPI':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSPI'].iloc[:,i], breakpoint)
        if market_bp == 'KOSDAQ':
            absolute_point = get_breakpoint(data[market_cat.iloc[:,i]=='KOSDAQ'].iloc[:,i], breakpoint)
        for j in range(len(breakpoint)+1):
            if j < len(breakpoint):
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] < absolute_point[j] and mask_np[k] == -1 else mask_np[k]
            else:
                for k in range(len(mask)):
                    mask_np[k] = j if data.iloc[k,i] > absolute_point[j-1] and mask_np[k] == -1 else mask_np[k]
        mask = mask * mask_np
        mask_list.append(mask)
    mask_df = pd.concat(mask_list, axis =1)
    return mask_df

def shift_date_quarter(data, num_of_quarters):
    # data의 date를 원하는 만큼 밀어줌.
    dates = data.columns
    years = [int(x[:4]) for x in dates]
    months = [int(x[4:6]) for x in dates]
    shifted_months = [x + num_of_quarters * 3 for x in months]
    shifted_years = [x + (shifted_months[i]-1) // 12 for i, x in enumerate(years)]
    shifted_months = [(x -1) % 12 +1 for x in shifted_months]
    dates = [str(shifted_years[i]) + str(shifted_months[i]).zfill(2)+ '30' if shifted_months[i] in [6, 9] else str(shifted_years[i]) + str(shifted_months[i]).zfill(2)+ '31' for i in np.arange(len(years))]
    data2 = copy.deepcopy(data)
    data2.columns = dates
    return data2
 
def get_factor_on_date_by_mask(mask, term = 'd', winsorize_limits = 0.01, weight = 'EW'):
    # @@@@@@@@@@@ option 추가해야할듯! : daily / weekly / monthly / quarterly 정도까지는 구현을 해놓아야할듯
    # monthly 계산할 때 첫 달은 어떻게 계산하는거지? 전달 말일 기준으로 계산하나? 아니면 해당 달 시초가 기준으로 계산하나?
    # -> 우선 첫 달은 시가 기준, 이후로는 전달 말일에 리밸런싱 하는 걸 기준으로 계산했음. 
    term_conservative = pd.Timestamp(mask.columns[1]) - pd.Timestamp(mask.columns[0]) - pd.Timedelta(days = 7)
    if pd.Timestamp(mask.columns[-1]) + term_conservative >  pd.Timestamp('20210630'):
        date_to = pd.Timestamp('20210630')
    else:
        date_to = pd.Timestamp(last_market_date((pd.Timestamp(mask.columns[-1]) + term_conservative).strftime('%Y%m%d')))
    date_from = pd.Timestamp(mask.columns[0])
    factor_days = (date_to - date_from).days
    factor_months = (date_to.year - date_from.year)*12 + (date_to.month - date_from.month)
    
    date_point = date_from
    # this is for storing the last date
    date_point_marked = None
    
    total_factor = []
    date_list = []
    
    if term == 'd':
        flag = -1
#         for _ in range(factor_days):
        for _ in tqdm.tqdm(range(10)):
            if pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
            date_point_marked = date_point
            first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            date_point = find_next_market_date(date_point_marked)
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            date_list.append(date_point)
            return_list = [[] for x in np.arange(mask.max()[0]+1)]
            value_list = [[] for x in np.arange(mask.max()[0]+1)]
            for i in range(len(first)):
                if first.iloc[i,0] in c_mask.index and c_mask.notna().loc[first.iloc[i,0]]:
                    if second.notna()[second['종목코드'] == first.iloc[i,0]].iloc[0,0]:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(second[second['종목코드'] == first.iloc[i,0]].iloc[0,3])
                    else:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(-0.99)
                    value_list[int(c_mask.loc[first.iloc[i,0]])].append(first.iloc[i,2])
                    
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
                    
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
                
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)
    elif term == 'm':
        flag = -1
        for _ in tqdm.tqdm(range(factor_months)):
            if flag +1 == mask.shape[1]:
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
                c_mask = c_mask.loc[list(filter(lambda x: x in first.index, c_mask[c_mask.notna()].index))]
            elif pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
                c_mask = c_mask.loc[list(filter(lambda x: x in first.index, c_mask[c_mask.notna()].index))]
            else:
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
            
            # 통일된 수식으로 flag가 바뀐 경우의 말일을 계산할 수 있음
            date_point = last_market_date(find_next_market_date(date_point_marked))
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            second.set_index('종목코드', inplace=True)
            date_list.append(date_point[:6])
            second_added = pd.DataFrame(index = list(set(c_mask.index) - set(second.index)), columns = ['종가', '시가총액', '등락률']).fillna(-99)
            second_added.iloc[:,1] = first.loc[list(set(c_mask.index) - set(second.index))].iloc[:,2]
            second = pd.concat([second, second_added])
            return_list = []
            value_list = []
            for i in range(int(mask.max()[0]+1)):
                return_list.append(second.loc[c_mask[c_mask == i].index].iloc[:,2].values)
                value_list.append(first.loc[c_mask[c_mask == i].index].iloc[:,1].values)
            c_mask.drop(list(set(c_mask.index) - set(second.index)), inplace=True)
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)

In [136]:
get_factor_on_date_by_mask(SMB_mask, 'm', weight = 'VW')

,0,1
201307,0.361063,0.158869
201308,-0.179827,0.619423
201309,-0.727365,-0.620385
201310,-0.571025,-0.860493
201311,0.129443,0.016084
...,...,...
202102,-2.130995,-1.804803
202103,0.428762,-0.093257
202104,-0.608465,-0.670366
202105,0.487541,0.438854


In [135]:
def get_factor_on_date_by_mask(mask, term = 'd', winsorize_limits = 0.01, weight = 'EW'):
    # @@@@@@@@@@@ option 추가해야할듯! : daily / weekly / monthly / quarterly 정도까지는 구현을 해놓아야할듯
    # monthly 계산할 때 첫 달은 어떻게 계산하는거지? 전달 말일 기준으로 계산하나? 아니면 해당 달 시초가 기준으로 계산하나?
    # -> 우선 첫 달은 시가 기준, 이후로는 전달 말일에 리밸런싱 하는 걸 기준으로 계산했음. 
    term_conservative = pd.Timestamp(mask.columns[1]) - pd.Timestamp(mask.columns[0]) - pd.Timedelta(days = 7)
    if pd.Timestamp(mask.columns[-1]) + term_conservative >  pd.Timestamp('20210630'):
        date_to = pd.Timestamp('20210630')
    else:
        date_to = pd.Timestamp(last_market_date((pd.Timestamp(mask.columns[-1]) + term_conservative).strftime('%Y%m%d')))
    date_from = pd.Timestamp(mask.columns[0])
    factor_days = (date_to - date_from).days
    factor_months = (date_to.year - date_from.year)*12 + (date_to.month - date_from.month)
    
    date_point = date_from
    # this is for storing the last date
    date_point_marked = None
    
    total_factor = []
    date_list = []
    
    if term == 'd':
        flag = -1
#         for _ in range(factor_days):
        for _ in tqdm.tqdm(range(10)):
            if pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
            date_point_marked = date_point
            first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            date_point = find_next_market_date(date_point_marked)
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            date_list.append(date_point)
            return_list = [[] for x in np.arange(mask.max()[0]+1)]
            value_list = [[] for x in np.arange(mask.max()[0]+1)]
            for i in range(len(first)):
                if first.iloc[i,0] in c_mask.index and c_mask.notna().loc[first.iloc[i,0]]:
                    if second.notna()[second['종목코드'] == first.iloc[i,0]].iloc[0,0]:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(second[second['종목코드'] == first.iloc[i,0]].iloc[0,3])
                    else:
                        return_list[int(c_mask.loc[first.iloc[i,0]])].append(-0.99)
                    value_list[int(c_mask.loc[first.iloc[i,0]])].append(first.iloc[i,2])
                    
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
                    
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
                
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)
    elif term == 'm':
        flag = -1
        for _ in tqdm.tqdm(range(factor_months)):
            if flag +1 == mask.shape[1]:
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
                c_mask = c_mask.loc[list(filter(lambda x: x in first.index, c_mask[c_mask.notna()].index))]
            elif pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
                c_mask = c_mask.loc[list(filter(lambda x: x in first.index, c_mask[c_mask.notna()].index))]
            else:
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
                first.set_index('종목코드', inplace = True)
            
            # 통일된 수식으로 flag가 바뀐 경우의 말일을 계산할 수 있음
            date_point = last_market_date(find_next_market_date(date_point_marked))
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            second.set_index('종목코드', inplace=True)
            date_list.append(date_point[:6])
            second_added = pd.DataFrame(index = list(set(c_mask.index) - set(second.index)), columns = ['종가', '시가총액', '등락률']).fillna(-99)
            second_added.iloc[:,1] = first.loc[list(set(c_mask.index) - set(second.index))].iloc[:,2]
            to_delete = list(set(c_mask.index) - set(second.index))
            second = pd.concat([second, second_added])
            return_list = []
            value_list = []
            for i in range(int(mask.max()[0]+1)):
                return_list.append(second.loc[c_mask[c_mask == i].index].iloc[:,2].values)
                value_list.append(first.loc[c_mask[c_mask == i].index].iloc[:,1].values)
            c_mask.drop(to_delete, inplace=True)
            if weight == 'VW':
                for i in range(len(return_list)):
                    # len을 곱해준 건 나중에 mean에서 EW와 같은 형태로 들어가게 하도록 하기 위함.
                    return_list[i] *= value_list[i]/ np.sum(value_list[i]) * len(value_list[i])
            # winsorize
            if winsorize_limits > 0:
                for i in range(len(return_list)):
                    return_list[i] = winsorize(return_list[i], limits = winsorize_limits)
            factor = [np.mean(x) for x in return_list]
            total_factor.append(factor)
        return pd.DataFrame(total_factor, index = date_list)

# Pfo Construction

In [2]:
def make_portfolio(pfo, factor, market_bp= 'All', breakpoint=[0.5]):
    # 여기서 앞에 들어가는 거는 마스크, 뒤에 들어가는 거는 실제값. 
    max_num = int(pfo.max()[0]+1)
    new_df = pd.DataFrame(index = pfo.index, columns = pfo.columns)
    for i in range(len(pfo.columns)):
        for j in range(max_num):
            get_index = list(filter(lambda x: x in factor.index, pfo.iloc[:,i][pfo.iloc[:,i]==j].index))
            pfo_mask = get_mask(pd.DataFrame(factor.loc[get_index].iloc[:,i]), get_market_category(pd.DataFrame(factor.loc[get_index].iloc[:,i])), market_bp, breakpoint)
            new_df.iloc[:,i].loc[pfo_mask.index] = pfo_mask.values[:,0]
    return new_df

def serialize_pfo(mask_list: list):
    # 사전식 배열. 앞에서부터 사전식으로 serialize하여 뱉음
    serial_num = [x.max().max()+1 for x in mask_list]
    for i in reversed(range(len(serial_num)-1)):
        serial_num[i] *= serial_num[i+1]
    serial_num += [1]
    serial_num = serial_num[1:]
    serial_elem = [x * mask_list[i] for i, x in enumerate(serial_num)]
    for i in range(len(serial_elem)-1):
        serial_elem[0] += serial_elem[i+1]
    return serial_elem[0]

# Regression 해볼 수 있도록 만들기

In [3]:
def get_market_return(mask, term = 'd'):
    term_conservative = pd.Timestamp(mask.columns[1]) - pd.Timestamp(mask.columns[0]) - pd.Timedelta(days = 7)
    if pd.Timestamp(mask.columns[-1]) + term_conservative >  pd.Timestamp('20210630'):
        date_to = pd.Timestamp('20210630')
    else:
        date_to = pd.Timestamp(last_market_date((pd.Timestamp(mask.columns[-1]) + term_conservative).strftime('%Y%m%d')))
    date_from = pd.Timestamp(mask.columns[0])
    factor_days = (date_to - date_from).days
    factor_months = (date_to.year - date_from.year)*12 + (date_to.month - date_from.month)
    
    date_point = date_from
    # this is for storing the last date
    date_point_marked = None
    
    total_return = []
    date_list = []
    
    if term == 'd':
        flag = -1
        date_point = find_next_market_date(date_point)
        for _ in tqdm.tqdm(range(factor_days)):
            if pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
            first = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            date_list.append(date_point)
            date_point = find_next_market_date(date_point)
            
            market_cap = first['시가총액'].sum()
            R = first['등락률'] * (first['시가총액'] / market_cap)
            total_return.append(R.fillna(0).sum())
            
        return pd.DataFrame(total_return, index = date_list)
    elif term == 'm':
        flag = -1
        for _ in tqdm.tqdm(range(factor_months)):
            if flag +1 == mask.shape[1]:
                c_mask = mask.iloc[:,flag]
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
            elif pd.Timestamp(find_next_market_date(date_point)) > pd.Timestamp(mask.columns[flag+1]):
                flag+=1
                c_mask = mask.iloc[:,flag]
                # 기준이 바뀌었을 경우에는 기간 시초가를 기준으로 계산
                date_point = find_next_market_date(date_point)
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '시가', '시가총액', '등락률']]
            else:
                date_point_marked = date_point
                first = get_market_data_on_date(date_point_marked, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            
            date_point = last_market_date(find_next_market_date(date_point_marked))
            second = get_market_data_on_date(date_point, print_error = False).loc[:,['종목코드', '종가', '시가총액', '등락률']]
            first.set_index('종목코드', inplace = True)
            second.set_index('종목코드', inplace = True)
            date_list.append(date_point[:6])
            return_list = [[] for x in np.arange(mask.max()[0]+1)]
            value_list = [[] for x in np.arange(mask.max()[0]+1)]
            market_cap = first['시가총액'].sum()
            try:
                R = second['등락률'] * (first['시가총액'] / market_cap)
            except KeyError:
                R = second['등락률'] * (first['시가총액'] / market_cap)
            
            total_return.append(R.replace([np.inf, -np.inf], np.nan).fillna(0).sum())
        return pd.DataFrame(total_return, index = date_list)

In [4]:
# risk free rate
os.chdir('C:\\Users\\lyoo9\\_Akros')
R_f = pd.read_excel('risk_free_rate.xls')
cd_91 = R_f.iloc[[1,6],1:]
cd_91 =cd_91.T.set_index(1)
cd_91.index = cd_91.index.map(lambda x:x[:-1])
cd_91.columns = ['cd_91']
os.chdir('C:\\Users\\lyoo9\\_System Trading\\Backend\\KRX_price')

In [ ]:
# pfo가 주어지면, 이제 해당 column별로 수익률을 회귀해야 함.
# 기간은 201307~202106으로 하자.

In [5]:
R_p = pfo.iloc[6:,0] - cd_91[:-4]
R_m = R_m.iloc[6:,0] - cd_91[:-4]
equity_factor = (equity_factor.iloc[:,0] - equity_factor.iloc[:,1]).iloc[6:0]

df = pd.DataFrame([R_p, R_m, equity_factor])

NameError: name 'pfo' is not defined

In [108]:
equity_factor = equity_factor[6:]

In [124]:
rf = cd_91[:-5].T.applymap(lambda x: float(x)) / 12

In [126]:
R_p = pfo.iloc[6:,0] - rf

In [128]:
R_m = get_market_return(get_mask(equity), term = 'm')

20130131
20130228
20130329
20130430
20130531
20130628
20130731
20130830
20130930
20131031
20131129
20140129
20140228
20140331
20140430
20140530
20140630
20140731
20140829
20140930
20141031
20141128
20150130
20150227
20150331
20150430
20150529
20150630
20150731
20150831
20150930
20151030
20151130
20160129
20160229
20160331
20160429
20160531
20160630
20160729
20160831
20160930
20161031
20161130
20170131
20170228
20170331
20170428
20170531
20170630
20170731
20170831
20170929
20171031
20171130
20180131
20180228
20180330
20180430
20180531
20180629
20180731
20180831
20180928
20181031
20181130
20190131
20190228
20190329
20190430
20190531
20190628
20190731
20190830
20190930
20191031
20191129
20200131
20200228
20200331
20200429
20200529
20200630
20200731
20200831
20200929
20201030
20201130



In [130]:
R_m = R_m.iloc[6:,0] - rf

In [140]:
df = pd.DataFrame([R_p.values[0], R_m.values[0], equity_factor.values])

In [142]:
df = df.T.set_index(equity_factor.index)

In [145]:
df.columns = ['pfo', 'market', 'equity']

In [146]:
df

,pfo,market,equity
201307,0.508809,-0.206166,0.676053
201308,0.656347,0.640605,-0.079115
201309,-1.160913,-0.912870,-0.329864
201310,-1.191863,-1.546809,0.049578
201311,0.473981,-0.216159,0.162706
...,...,...,...
202102,-1.814539,-2.777507,-0.439670
202103,-0.213161,-0.370505,0.046582
202104,-0.520153,-0.829644,-0.029250
202105,0.299991,0.378394,-0.069121


# 모듈화하기

In [ ]:
# 우선, mask에 따른 시장수익률과 무위험 수익률을 구해야함. 

In [ ]:
# mask가 들어가면 시장수익률을 뱉어내도록 되어있음. 
# 그러면, mask를 받아서 시장수익률을 뱉어내고, 무위험 수익률은 어떡하지? -> 이것도 하나 만들자.

In [5]:
def get_risk_free_rate(pfo, risk_free):
    return risk_free.loc[pfo.index]

def get_ols_model(mask, pfo_column, risk_free, *args):
    R_m = get_market_return(mask, term = 'm').squeeze()
    R_f = (get_risk_free_rate(pfo_column, risk_free).astype(float)/12).squeeze()
    df = pd.DataFrame([(pfo_column - R_f.T).values.squeeze(), (R_m - R_f).values.squeeze()] + [x.values.squeeze() for x in args]).T
    df.columns = ['pfo', 'market'] + ['factor' + str(i) for i, x in enumerate(args)]
    var_str = ''
    for i in range(len(df.columns)-2):
        var_str += ' + ' + ['factor' + str(i) for i, x in enumerate(args)][i]
    model = ols(formula = 'pfo ~ market' + var_str,data = df).fit()
    return model

In [122]:
model = get_ols_model(get_mask(equity), pfo.iloc[:,0], cd_91, equity_factor.iloc[:,0] - equity_factor.iloc[:,1])

In [123]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    pfo   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     136.5
Date:                Wed, 16 Feb 2022   Prob (F-statistic):           7.76e-20
Time:                        21:01:50   Log-Likelihood:                -21.530
No. Observations:                  48   AIC:                             49.06
Df Residuals:                      45   BIC:                             54.67
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0989      0.058     -1.711      0.094      -0.215       0.018
market         0.7289      0.087      8.354      0.000       0.553       0.905
factor0        1.0587      0.082     12.875      0.000       0.893       1.224
==============================================================================
Omnibus:                        0.759   Durbin-Watson:                   1.806
Prob(Omnibus):                  0.684   Jarque-Bera (JB):                0.781
Skew:                          -0.093   Prob(JB):                        0.677
Kurtosis:                       2.403   Cond. No.                         1.66
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 이제 FF3 구현해보자. 

In [6]:
Market_cap = get_market_cap_for_period('20121231', '20191231')

equity = get_item_for_period('ifrs_Equity', '20121231', '20191231')

BEME = equity/Market_cap[0]

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [170]:
Market_cap2 = get_market_cap_for_period('20121231', '20191231')

In [7]:
Market_cap = get_market_cap_for_period('20130630', '20200630')

In [156]:
# SMB
SMB_mask = get_mask(Market_cap[0], Market_cap[1], market_bp = 'KOSPI')

SMB = get_factor_on_date_by_mask(SMB_mask, term = 'm', winsorize_limits = 0.01, weight = 'VW')

In [138]:
# HML
BEME_shifted = shift_date_quarter(BEME, 2)

HML_mask = get_mask(BEME_shifted, get_market_category(BEME_shifted), market_bp = 'KOSPI', breakpoint = [0.3, 0.7])

HML = get_factor_on_date_by_mask(HML_mask, term = 'm', winsorize_limits = 0.01, weight = 'VW')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [150]:
# RMW 계산하기
# (영업이익 - 이자비용) / 총자산
Total_asset = get_item_for_period('ifrs_Assets', '20121231', '20191231')
fcost1 = get_item_for_period('ifrs_FinanceCosts', '20121231', '20191231')
fcost2 = get_item_for_period('dart_InterestExpenseFinanceExpense', '20121231', '20191231')
fcost = pd.concat([fcost1, fcost2], axis = 1).groupby(level=0, axis=1).last()
OI = get_item_for_period('dart_OperatingIncomeLoss', '20121231', '20191231')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [151]:
RMW_base = (OI - fcost) / Total_asset

RMW_base_shifted = shift_date_quarter(RMW_base, 2)

RMW_mask = get_mask(RMW_base_shifted, get_market_category(RMW_base_shifted), market_bp = 'KOSPI', breakpoint = [0.3, 0.7])

RMW = get_factor_on_date_by_mask(RMW_mask, term = 'm', winsorize_limits = 0.01, weight = 'VW')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [152]:
# CMA 계산하기

Total_asset = get_item_for_period('ifrs_Assets', '20111231', '20181231')
pnl = get_item_for_period('ifrs_ProfitLoss', '20121231', '20191231')
Total_asset_shifted = shift_date_quarter(Total_asset, 4)
CMA_base = pnl / Total_asset_shifted
CMA_base_shifted = shift_date_quarter(CMA_base, 2)
CMA_mask = get_mask(CMA_base_shifted, get_market_category(CMA_base_shifted), market_bp = 'KOSPI', breakpoint = [0.3, 0.7])
CMA = get_factor_on_date_by_mask(CMA_mask, term = 'm', winsorize_limits = 0.01, weight = 'VW')

C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:1229: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  v = np.array(v, copy=False)
C:\Users\lyoo9\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\lyoo9\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [153]:
# HML pfo
hml_pfo_mask = make_portfolio(get_mask(Market_cap[0], Market_cap[1], breakpoint = [0.2, 0.4, 0.6, 0.8]), BEME_shifted, breakpoint = [0.2, 0.4, 0.6, 0.8])

pfo25 = serialize_pfo([get_mask(Market_cap[0], Market_cap[1], breakpoint = [0.2, 0.4, 0.6, 0.8]), hml_pfo_mask])

pfo_return = get_factor_on_date_by_mask(pfo25, term = 'm', weight = 'EW')

In [154]:
pfo_return

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
201307,-0.158800,0.465385,-0.047200,0.198077,0.394231,0.382826,-0.017447,0.336596,1.142766,-0.025957,...,1.517885,0.948302,1.002453,0.154151,0.292075,1.023571,1.004821,1.049643,0.240714,0.640702
201308,-0.588400,-0.766923,-0.308800,-0.178077,-0.232692,0.343043,0.453830,-0.176596,0.041489,-0.679787,...,0.834038,-0.145472,-0.056981,0.257170,0.224528,0.428393,-0.096964,0.513393,0.290536,0.445614
201309,0.659200,-1.704615,0.080400,-0.435000,-0.202308,-1.063261,-0.549787,-0.520426,-0.954255,-0.405319,...,-0.569231,-0.537547,-0.857736,0.529245,-0.975283,-0.804107,-0.816607,-0.358929,-0.620714,-0.388421
201310,0.868000,0.683462,-0.957600,0.464615,-0.065000,-0.321522,-0.891064,-0.415106,-0.447234,-0.441702,...,-0.687115,-0.924906,-0.158491,-0.630000,-0.780189,-1.416964,-1.155179,-0.721607,-0.654107,-0.632281
201311,-0.447200,-0.266538,-0.621600,-0.692308,-0.970385,0.216087,0.301489,0.193617,0.080213,-0.365745,...,0.538077,0.324906,0.429434,0.283208,-0.097547,0.598036,-0.150179,0.141607,0.599643,0.524737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202102,-1.045429,-1.639722,-2.326389,-1.318889,-0.694167,-2.358378,-2.738514,-2.381467,-2.500135,-1.167733,...,-1.496154,-1.847848,-3.195128,-2.468228,-1.789620,-1.862025,-1.741000,-2.070125,-1.552250,-1.869877
202103,0.260571,0.668056,0.776944,0.978611,-0.001111,-0.294459,-0.352703,0.133600,0.725541,1.105600,...,0.304231,-0.582278,0.925897,0.296076,1.706582,-1.041899,-0.344750,0.368375,0.033375,0.652222
202104,0.224000,0.261667,-1.912778,-1.054722,0.363333,-1.495405,-0.995135,-0.793067,-0.318649,-0.770533,...,-0.177564,-1.242911,-0.424487,-0.145190,-0.685063,-0.760253,-0.225250,-0.408750,-0.624625,-1.261481
202105,0.017714,-0.098056,0.897778,1.030278,1.590278,0.559324,1.547838,1.147867,0.782027,0.864533,...,0.873846,0.829620,0.873205,0.841392,0.777975,0.571772,0.960750,0.722250,0.146125,0.798642


In [155]:
coef = []
tvalues = []
r = []
for i in range(25):
    model = get_ols_model(pfo25, pfo_return.iloc[:,i], cd_91, SMB.iloc[:,0] - SMB.iloc[:,1], HML.iloc[:,2] - HML.iloc[:,0], RMW.iloc[:,2] - RMW.iloc[:,0], CMA.iloc[:,2] - CMA.iloc[:,0])
    coef.append(model.params)
    tvalues.append(model.tvalues)
    r.append(model.rsquared)

In [158]:
pd.concat(coef, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Intercept,0.098523,0.050380,0.119707,0.069258,0.067941,0.021790,0.059728,0.074733,0.142079,0.080370,...,0.041056,0.018881,0.059991,0.074199,0.071287,0.073200,0.005212,0.060044,0.018967,-0.012631
market,0.604796,0.474850,0.848215,0.377337,0.517203,0.667836,0.773877,0.828704,0.740274,0.641141,...,0.745625,0.745060,0.850235,0.754554,0.600960,0.733733,0.720748,0.669154,0.729663,0.546406
factor0,1.208854,0.894404,0.932805,1.165262,0.612401,1.158244,1.071122,1.106607,0.985703,0.810176,...,0.754481,0.858425,1.010159,0.802770,0.521614,0.028409,0.260265,0.171157,0.202583,-0.023334
factor1,-0.027678,0.257932,0.356774,0.447777,0.202704,-0.072837,0.113292,0.124794,0.099456,0.392487,...,-0.532027,-0.239083,0.140296,0.182087,0.422214,-0.776057,-0.218224,-0.100335,0.267804,0.509029
factor2,-0.310040,0.218767,-0.457007,0.495374,0.103570,0.285141,0.376678,0.246228,0.276278,0.370098,...,-0.076042,0.098924,0.107611,0.331486,0.295136,0.191418,0.443813,0.356415,0.159085,0.430463
factor3,0.997745,-0.342577,0.463993,-0.267625,-0.208285,0.000500,-0.299026,-0.001934,-0.178130,-0.369356,...,-0.548602,-0.391246,-0.143072,-0.321735,-0.371898,-1.071489,-0.697698,-0.740846,-0.340997,-0.732618


In [163]:
pd.concat(tvalues, axis=1)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
Intercept,1.144338,0.728494,1.344542,0.947818,0.914008,0.369481,1.157930,1.534483,2.781025,1.430106,...,0.596463,0.366414,1.245542,1.590644,1.346815,1.252161,0.105299,1.207154,0.427658,-0.275734
market,5.422336,5.300095,7.353962,3.986051,5.370759,8.740953,11.580690,13.134421,11.184760,8.806210,...,8.361505,11.160661,13.625979,12.486006,8.764019,9.688260,11.240397,10.384414,12.699094,9.207374
factor0,7.544079,6.948888,5.629382,8.568234,4.426546,10.552209,11.157223,12.208422,10.366560,7.745851,...,5.889344,8.950670,11.268681,9.246530,5.294948,0.261108,2.825320,1.848862,2.454187,-0.273689
factor1,-0.192218,2.230087,2.396064,3.664077,1.630518,-0.738462,1.313258,1.532129,1.164008,4.175904,...,-4.621540,-2.774198,1.741662,2.334001,4.769574,-7.937622,-2.636270,-1.206138,3.610417,6.644331
factor2,-1.242570,1.091525,-1.771181,2.339222,0.480764,1.668297,2.519752,1.744511,1.865975,2.272361,...,-0.381189,0.662408,0.770919,2.452012,1.924002,1.129836,3.094015,2.472502,1.237672,3.242507
factor3,3.375210,-1.442743,1.517856,-1.066703,-0.816087,0.002470,-1.688400,-0.011566,-1.015485,-1.914188,...,-2.321266,-2.211324,-0.865142,-2.008791,-2.046377,-5.338245,-4.105527,-4.337982,-2.239261,-4.658016


In [160]:
r

[0.5640928533838516,
 0.7351187660600726,
 0.6623672729374508,
 0.7284448194396365,
 0.6214191179872637,
 0.8315968680481619,
 0.8901411076523956,
 0.894444023587854,
 0.8730396285149936,
 0.83025202217042,
 0.8316813131898545,
 0.9043137473116359,
 0.9008976187500319,
 0.8985763583806846,
 0.8680417411465412,
 0.8264100395853473,
 0.882436692769221,
 0.9026549985018738,
 0.8865023220790371,
 0.8002107395403889,
 0.8379444683067706,
 0.8395781932716608,
 0.8223831808787917,
 0.8384647945560972,
 0.8127217552955495]